# Import Libraries

In [1]:
import platform
import time
import os
os.chdir('/hri/localdisk/Amir/AttentiveSupport/build')
print(os.getcwd())
import json
import numpy as np
import requests
url = "http://172.26.1.195:5000"
import sys


if platform.system() == "Linux":
    sys.path.append("lib")
elif platform.system() == "Windows":
    sys.path.append("bin")

from pyAffaction import *
import random
setLogLevel(-1)

/hri/localdisk/Amir/AttentiveSupport/build


In [2]:
def map_sim2ccdp(pos):
    n_d = pos.shape[-1]
    init_shape = pos.shape
    pos = pos.reshape([-1,n_d])
    if n_d == 2:
        pos_ccdp = pos[:,[1,0]]/0.7
    elif n_d == 3:
        pos_ccdp = pos[:,[1,0,2]]/0.7
    pos_ccdp[:,0] *= -1
    return pos_ccdp.reshape(init_shape)

def map_ccdp2sim(pos):
    n_d = pos.shape[-1]
    init_shape = pos.shape
    pos = pos.reshape([-1,n_d])
    if n_d == 2:
        pos_sim = pos[:,[1,0]]*0.7
    elif n_d == 3:
        pos_sim = pos[:,[1,0,2]]*0.7
    elif n_d == 4:
        pos_sim = pos[:,[1,0,2,3]]
        pos_sim[:,:-1] *= 0.7
    pos_sim[:,1] *= -1
    return pos_sim.reshape(init_shape)

In [7]:
# Get Object Locations
table_pos = np.array([-0.1, 0.2, 1.10])

batch_size = 64
n_history = 2
n_pred = 8
applied = 8
obj_height = 0.05

box_locations = [
    np.array([[0,0.35]]),
    np.array([[0.21,0.0]]),
    np.array([[0,-0.35]])
]


def get_obj_locations(sim):
    states = json.loads(sim.get_state())['entities']
    positions = []
    for state in states:
        obj_color = state["color"]
        if obj_color == 'GREEN':
            psotions_i = state["position"]
            rel_table_pos = psotions_i - table_pos
            # print(psotions_i, rel_table_pos)
            if rel_table_pos[2] > -0.5:
                positions.append(rel_table_pos[:2])
    positions_array = np.array(positions)
    return positions_array



def get_obj_locations_3D(sim):
    states = json.loads(sim.get_state())['entities']
    positions = []
    for state in states:
        obj_color = state["color"]
        if obj_color == 'GREEN':
            psotions_i = state["position"]
            rel_table_pos = psotions_i - table_pos
            # print(psotions_i, rel_table_pos)
            if rel_table_pos[2] > -0.5:
                positions.append(rel_table_pos[:3])
    positions_array = np.array(positions)
    return positions_array

# Filter those that are located in a box
def mask_in_positions(obj_positions, box_positions):
    dists = np.zeros([len(obj_positions), len(box_positions)])
    for i, box in enumerate(box_positions):
        dists[:,i] = np.linalg.norm(obj_positions - box, axis = 1)
    closest_boxes = np.min(dists, axis = 1)
    check_reached = (closest_boxes < 0.13)
    not_reached_idx = [i for i in range(len(obj_positions)) if check_reached[i] == 0]
    masked_pos = obj_positions[not_reached_idx,:]
    return masked_pos


def count_available_obj(sim):
    obj_locations = get_obj_locations(sim)
    available_obj = mask_in_positions(obj_locations,box_locations)
    return available_obj.shape[0]

# Find the closest object
def closest_object(sim):
    end_effector_pos = get_end_effector_pos(sim)
    obj_locations = get_obj_locations(sim)
    available_objects = mask_in_positions(obj_locations, box_locations)
    ee_pos_2d = end_effector_pos[:2].reshape([1,-1])
    dists = np.linalg.norm(available_objects - ee_pos_2d, axis = 1)
    closest_obj = np.argmin(dists, axis = 0)
    return available_objects[closest_obj,:]

def get_end_effector_pos(sim):
    states = json.loads(sim.get_state())
    for gripper in states["agents"][0]["manipulators"]:
        if gripper["name"] == "hand_left_robot":
            pos = gripper["position"]
            relative_pos_table = pos.copy()
            relative_pos_table -= table_pos
            return relative_pos_table



# Plan traj
v_des = 0.5
def plan_traj(sim):
    obj_loc = closest_object(sim)
    obj_pos_3D = np.concatenate([obj_loc,[0.04]])
    ee_pos = get_end_effector_pos(sim)
    box_location = box_locations[2].reshape([-1])
    box_loc_3D = np.concatenate([box_location,[0.0]])

    p0 = ee_pos.copy()
    p0 [2] -= 0.1
    p1 = obj_pos_3D.copy()
    p1[2] += 0.15
    p2 = obj_pos_3D.copy()
    p2[2] -= 0.00
    p3 = p1.copy()
    p4 = box_loc_3D.copy()
    p4[2] += 0.15

    all_trajs = np.zeros([7,5])
    all_trajs[0,0] = 3
    all_trajs[0,1:4] = p0
    all_trajs[0,4] = 0

    delta_move = np.linalg.norm(p1 - p0)
    all_trajs[1,0] = all_trajs[0,0] + delta_move/v_des + .1
    all_trajs[1,1:4] = p1
    all_trajs[1,4] = 0

    delta_move = np.linalg.norm(p2 - p1)
    all_trajs[2,0] = all_trajs[1,0] + delta_move/v_des + .1
    all_trajs[2,1:4] = p2
    all_trajs[2,4] = 0

    all_trajs[3,0] = all_trajs[2,0] + 1
    all_trajs[3,1:4] = p2
    all_trajs[3,4] = 1

    delta_move = np.linalg.norm(p3 - p2)
    all_trajs[4,0] = all_trajs[3,0] + delta_move/v_des + .1
    all_trajs[4,1:4] = p3
    all_trajs[4,4] = 1

    delta_move = np.linalg.norm(p4 - p3)
    all_trajs[5,0] = all_trajs[4,0] + delta_move/v_des + .1
    all_trajs[5,1:4] = p4
    all_trajs[5,4] = 1

    all_trajs[6,0] = all_trajs[5,0] + 1
    all_trajs[6,1:4] = p4
    all_trajs[6,4] = 0


    np.savetxt('test_robot_traj.txt', all_trajs, fmt='%.3f', delimiter=' ')



def plan_traj_ccdp(sim, failure_modes = []):
    obj_loc_sim = closest_object(sim)
    ee_pos_sim = get_end_effector_pos(sim)
    
    obj_loc = map_sim2ccdp(obj_loc_sim)
    ee_pos = map_sim2ccdp(ee_pos_sim)

    payload = {"jsonrpc": "2.0",
                "method": "get_traj",
                "params": [ee_pos.tolist(),obj_loc.tolist(), failure_modes],
                "id": 1}


    response = requests.post(url, json=payload)
    result = response.json()
    output_array = np.array(result['result'])
    applied_future = output_array[0,:,:]
    traj_sim = map_ccdp2sim(applied_future)
    return traj_sim



def prepare_traj_robot(traj,max_des_height = 0.2, min_des_height = 0.03):
    v_des = 0.1
    traj_time_stamped = np.zeros([1, traj.shape[1]+1])
    traj_time_stamped[0,1:4] = traj[0,:3]
    traj_time_stamped[0,0] = 1
    traj_time_stamped[0,3] = max_des_height

    min_height = traj[1:,2].min()
    max_height = traj[1:,2].max()
    mean_height = (min_height + max_height)/2
    traj_i = np.zeros([1,5])
    for i in range(1, traj.shape[0]):
        traj_i[0,1:3] = traj[i,:2] 
        traj_i[0,3] = (traj[i,2] < mean_height) * min_des_height + (traj[i,2] > mean_height) * max_des_height 
        dist_i = np.linalg.norm(traj_i[0,1:4] - traj_time_stamped[-1,1:4])
        delta_t = dist_i/v_des + 0.1
        traj_i[0,0] = traj_time_stamped[-1,0] + delta_t
        if traj[i,3] == 1 and traj[i-1,3] == 0:
            traj_time_stamped = np.concatenate([traj_time_stamped, traj_i], axis = 0)
            traj_i[0,4] = 1
            traj_i[0,0] += 1
            traj_time_stamped = np.concatenate([traj_time_stamped, traj_i], axis = 0)
        else:
            traj_time_stamped = np.concatenate([traj_time_stamped, traj_i], axis = 0)
    traj_i[0,4] = 0
    traj_i[0,0] += 1
    traj_time_stamped = np.concatenate([traj_time_stamped, traj_i], axis = 0)
    np.savetxt('test_robot_traj.txt', traj_time_stamped, fmt='%.3f', delimiter=' ')



def find_closest_box(pos_2d):
    min_dist = 1000
    min_id = -1
    for i in range(len(box_locations)):
        dist_i = np.linalg.norm(pos_2d - box_locations[i])
        if dist_i < min_dist:
            min_id = i
            min_dist = dist_i
    box_loc_sim = box_locations[min_id]
    return map_sim2ccdp(box_loc_sim)



In [8]:
sim = LlmSim()
sim.noTextGui = True
sim.unittest = False
sim.speedUp = 1
sim.noLimits = False
sim.verbose = False
sim.configDirectory = "config/xml/CCDP"
sim.xmlFileName = "g_iros25b.xml"
sim.addComponentArgument("-physics Bullet -physics_config config/xml/CCDP/physics_ccdp.xml")
#sim.addComponentArgument("-landmarks_zmq -yolo_tracking -landmarks_connection tcp://localhost:5556")
#sim.addComponentArgument("-landmarks_zmq2 -aruco_tracking2 -landmarks_connection2 tcp://localhost:5555")

sim.init(True)
sim.run()





********************************************************************************
* LLMSim initialized
********************************************************************************


In [10]:
sim.reset()
# time.sleep(2)

[AffAction/examples/ExampleActionsECS.cpp(1043)]: SetGazeTarget(hand_base_left)
qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""


In [11]:

count_av_apple = count_available_obj(sim)
prev_count = count_av_apple
failure_modes = []
while count_av_apple > 0 and len(failure_modes)<3:
    # print("count_available_obj(sim) ")
    
    # torch.manual_seed(0)
    # random.seed(0)
    # np.random.seed(0)
    traj = plan_traj_ccdp(sim, failure_modes)
    print("traj is ready")
    prepare_traj_robot(traj)
    sim.createActionFile()
    action_fb = sim.plan_fb("load action_iros.xml; pose default_top")
    if action_fb[:8] == "SUCCESS":
        time.sleep(1)
        count_av_apple = count_available_obj(sim)
        if count_av_apple >= prev_count:
            final_goal = traj[-1,:2]
            closest_box = find_closest_box(final_goal)
            print("failure occured for target:", closest_box)
            failure_modes.append(closest_box.tolist())
        prev_count = count_av_apple
    else:
        print("Closest object is reachable")
        time.sleep(5)
    

traj is ready
failure occured for target: [[-0.5  0. ]]
traj is ready
traj is ready
traj is ready
traj is ready


In [19]:
a = sim.plan_fb("load action_iros.xml; pose default_top")

In [29]:

    print(a)

SUCCESS


In [10]:
sim.createActionFile()

[Rcs/src/RcsGraphics/RcsViewer.cpp: handle(1902)]: Setting debug level to 1


In [11]:

sim.plan_fb("load action_iros.xml; pose default_top")

-1 - 0   thread: 10


[AffAction/src/ActionSequence.cpp(165)]: Received sequence with 4 parameters (two are expected).
[AffAction/src/PredictionTree.cpp(1197)]: Planning with 12 threads - early exit is TRUE
[AffAction/src/ActionFromXML.cpp(91)]: Creating action: iros25
[AffAction/src/PredictionTree.cpp(1164)]: Stopping thread 0 - 0 running
[AffAction/src/PredictionTree.cpp(1029)]: Started thread 9 parent: 0
[AffAction/src/PredictionTree.cpp : expand(1045)]: duration is 10.000000
[AffAction/src/PredictionTree.cpp(1210)]: Started threads: 10 stopped threads: 10
[AffAction/examples/ExampleActionsECS.cpp(1362)]: Solution 0 is NOT SUCCESSFUL
[AffAction/examples/ExampleActionsECS.cpp(1388)]: Found 1 error messages of solution 0
[AffAction/examples/ExampleActionsECS.cpp(1391)]: ACTION: 'load action_iros.xml' ERROR: 'Collision problem' REASON: 'The thumb_left collides with the table' SUGGESTION: 'Try to get the table out of the way' DEVELOPER: 'Collision detected at pair-idx 0 between thumb_left and table AffAction

'No solution found:\n  Issue 0: The thumb_left collides with the table Suggestion: Try to get the table out of the way\n'

In [28]:
a = get_obj_locations(sim)
c = mask_in_positions(a,box_locations)
print(c)

[[-0.20000103  0.29999898]
 [-0.20000001 -0.29998241]]


In [20]:
dists = np.zeros([len(a), len(box_locations)])
for i, box in enumerate(box_locations):
    dists[:,i] = np.linalg.norm(a - box, axis = 1)
print(dists)
# check_reached = (dists < 0.1).sum(axis = 1)
# not_reached_idx = [i for i in range(len(obj_positions)) if check_reached[i] == 0]
# masked_pos = obj_positions[not_reached_idx,:]


[[2.06156524e-01 5.08035659e-01 6.80072854e-01]
 [4.08165198e-01 1.43580175e-06 4.08167355e-01]
 [6.80061952e-01 5.08028289e-01 2.06158231e-01]]


In [57]:
a = json.loads(sim.get_state())

In [106]:
a['agents'][0]["manipulators"][0]["position"]

[-0.22176707227800802, 0.3000000013647248, 1.387328085947795]

In [105]:
get_end_effector_pos(sim)

array([-0.12176707,  0.1       ,  0.28732809])

In [108]:
get_obj_locations(sim)

array([[-2.00001026e-01,  2.99998982e-01],
       [ 2.09999308e-01,  1.25786304e-06],
       [-2.00000012e-01, -2.99867919e-01]])

In [115]:
obj_loc = closest_object(sim)
np.concatenate([obj_loc, [1.1]])

array([-0.20000103,  0.29999898,  1.1       ])

In [8]:
b = [np.array([1,2,3]),np.array([4,5,6])]
b_list = []
for arr in b:
    b_list.append(arr.tolist())
print(len(b_list))
b= []
for lis in b_list:
    b.append(np.array(lis))

2


In [11]:
import trimesh
import shapely.geometry as geometry

# Define a 2D triangle using Shapely
triangle = geometry.Polygon([(-0.1328, 0), (0.1328, 0), (0.1328, -0.11)])  # Equilateral triangle

# Extrude the triangle along the Z-axis to create a 3D shape
height = .01  # Adjust this to change the height
mesh = trimesh.creation.extrude_polygon(triangle, height=height)

mesh.apply_transform(trimesh.transformations.rotation_matrix(angle=-1.5708, direction=[1, 0, 0]))
# Export the mesh to an STL file
mesh.export('../src/Smile/src/AffAction/config/xml/CCDP/triangle.stl')

print("3D triangle (triangular prism) exported as 'triangle_prism.stl'")

3D triangle (triangular prism) exported as 'triangle_prism.stl'


In [6]:
3**0.5/2

0.8660254037844386